In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# 파일 경로 설정
file_path = '/content/drive/MyDrive/KOPIS/final_merged(2023)/merged_venue_FC222253-01.xlsx'

# Excel 파일 읽기
df = pd.read_excel(file_path)

In [ ]:
# 공연코드가 'PF443217'이고 공연일시가 '2023-11-19 23:00'인 공연만 제외하고 필터링
df = df[(df['공연코드'] == 'PF443217') & (df['공연일시'] != '2023-11-19 23:00')]

# 결과 확인
print(df)

         전송사업자코드 전송사업자명    공연시설코드       시설특성  개관연도  \
0       SE000201   인터파크  FC222253  민간(대학로 외)  2011   
1       SE000201   인터파크  FC222253  민간(대학로 외)  2011   
2       SE000201   인터파크  FC222253  민간(대학로 외)  2011   
3       SE000201   인터파크  FC222253  민간(대학로 외)  2011   
4       SE000201   인터파크  FC222253  민간(대학로 외)  2011   
...          ...    ...       ...        ...   ...   
651279  SE000201   인터파크  FC222253  민간(대학로 외)  2011   
651280  SE000201   인터파크  FC222253  민간(대학로 외)  2011   
651281  SE000201   인터파크  FC222253  민간(대학로 외)  2011   
651282  SE000201   인터파크  FC222253  민간(대학로 외)  2011   
651283  SE000201   인터파크  FC222253  민간(대학로 외)  2011   

                                    주소 편의시설_레스토랑 여부 편의시설_카페 여부 편의시설_편의점 여부  \
0       서울특별시 용산구 이태원로 294 (한남동) 블루스퀘어            Y          Y           N   
1       서울특별시 용산구 이태원로 294 (한남동) 블루스퀘어            Y          Y           N   
2       서울특별시 용산구 이태원로 294 (한남동) 블루스퀘어            Y          Y           N   
3       서울특별시 용산구 이태원로 294 (한남동) 블루스퀘어 

## 1. 타겟값 생성

- 취소 여부 결정




새로만들어진 칼럼: 예매일시,취소일시,예매후남은일자,취소후남은일자,취소여부

칼럼 수 : 기본68 + 5 = 73

In [ ]:
# '공연일시'와 '예매/취소일시' 칼럼을 datetime 객체로 변환
df['공연일시'] = pd.to_datetime(df['공연일시'])
df['예매/취소일시'] = pd.to_datetime(df['예매/취소일시'])

# '예매일시'와 '취소일시' 칼럼 추가 및 값 할당
df['예매일시'] = df.apply(lambda row: row['예매/취소일시'] if row['예매/취소구분'] == 1 else pd.NaT, axis=1)
df['취소일시'] = df.apply(lambda row: row['예매/취소일시'] if row['예매/취소구분'] == 2 else pd.NaT, axis=1)

# 그룹화하여 예매일자와 취소일자를 각각 추출
df['예매일시'] = df.groupby('입장권고유번호')['예매일시'].transform('min')
df['취소일시'] = df.groupby('입장권고유번호')['취소일시'].transform('min')

# 예매후남은일자와 취소후남은일자 계산
df['예매후남은일자'] = (df['공연일시'] - df['예매일시']).dt.days
df['취소후남은일자'] = (df['공연일시'] - df['취소일시']).dt.days

# 취소여부 칼럼 생성
df['취소여부'] = df['취소후남은일자'].apply(lambda x: 1 if pd.notnull(x) else 0)

# '예매/취소구분' 값이 2인 행을 삭제
df = df[df['예매/취소구분'] != 2]


# 결과 출력
print(df)



         전송사업자코드 전송사업자명    공연시설코드       시설특성  개관연도  \
0       SE000201   인터파크  FC222253  민간(대학로 외)  2011   
1       SE000201   인터파크  FC222253  민간(대학로 외)  2011   
2       SE000201   인터파크  FC222253  민간(대학로 외)  2011   
3       SE000201   인터파크  FC222253  민간(대학로 외)  2011   
4       SE000201   인터파크  FC222253  민간(대학로 외)  2011   
...          ...    ...       ...        ...   ...   
651279  SE000201   인터파크  FC222253  민간(대학로 외)  2011   
651280  SE000201   인터파크  FC222253  민간(대학로 외)  2011   
651281  SE000201   인터파크  FC222253  민간(대학로 외)  2011   
651282  SE000201   인터파크  FC222253  민간(대학로 외)  2011   
651283  SE000201   인터파크  FC222253  민간(대학로 외)  2011   

                                    주소 편의시설_레스토랑 여부 편의시설_카페 여부 편의시설_편의점 여부  \
0       서울특별시 용산구 이태원로 294 (한남동) 블루스퀘어            Y          Y           N   
1       서울특별시 용산구 이태원로 294 (한남동) 블루스퀘어            Y          Y           N   
2       서울특별시 용산구 이태원로 294 (한남동) 블루스퀘어            Y          Y           N   
3       서울특별시 용산구 이태원로 294 (한남동) 블루스퀘어 

## 2. 데이터 전처리


2.1.   종속관계인 칼럼의 경우 코드만 남기고 명은 삭제
- ('전송사업자코드', '전송사업자명')  >  코드에서 SE는 삭제학고 숫자만 사용  
('예매/취소방식코드','예매/취소방식명(관리시스템)', '예매/취소방식명(전송처)')  
('할인종류코드', '할인종류명(관리시스템)', '할인종류명(전송처)')  
('결제수단코드', '결제수단명(관리시스템)','결제수단명(전송처)')  
취소후남은일자 삭제
8개 삭제  
73 - 8 = 65




In [ ]:
# 드랍할 열 이름 리스트
columns_to_drop = ['전송사업자명',
                   '예매/취소방식명(관리시스템)', '예매/취소방식명(전송처)',
                   '할인종류명(관리시스템)', '할인종류명(전송처)',
                   '결제수단명(관리시스템)','결제수단명(전송처)',
                    '취소후남은일자']


# 열 삭제
df = df.drop(columns=columns_to_drop)


2.2 공연장으로 분류하였기에 공연장코드에 종속적인 칼럼 모두 삭제
'공연장코드' > 일단 살림
- 삭제(22개)

 '공연장코드','공연시설코드', '시설특성', '개관연도', '주소', '편의시설_레스토랑 여부', '편의시설_카페 여부', '편의시설_편의점 여부', '편의시설_놀이방 여부', '편의시설_수유실 여부', '장애인시설_주차장 여부', '장애인시설_화장실 여부', '장애인시설_경사로 여부', '장애인시설_전용엘리베이터 여부', '주차시설_자체 여부', '주차시설_공영 여부','좌석수', '장애인석', '무대시설_오케스트라피트 여부', '무대시설_연습실 여부', '무대시설_분장실 여부', '무대시설_무대넓이'  
65 - 22 = 43

In [ ]:
# 드랍할 열 이름 리스트
columns_to_drop = ['공연장코드','공연시설코드', '시설특성', '개관연도', '주소', '편의시설_레스토랑 여부',
                   '편의시설_카페 여부', '편의시설_편의점 여부', '편의시설_놀이방 여부', '편의시설_수유실 여부',
                   '장애인시설_주차장 여부', '장애인시설_화장실 여부', '장애인시설_경사로 여부',
                   '장애인시설_전용엘리베이터 여부', '주차시설_자체 여부', '주차시설_공영 여부','좌석수',
                   '장애인석', '무대시설_오케스트라피트 여부', '무대시설_연습실 여부', '무대시설_분장실 여부', '무대시설_무대넓이']

# 열 삭제
df = df.drop(columns=columns_to_drop)

2.3 공연코드에 종속적인 칼럼 공연코드만 남기고 삭제  
'공연코드' > 공연별 라벨링 필요(추가조사)  
-삭제(21개)  
'공연회차','공연시작일자', '공연종료일자', '소요시간', '장르명', '세부장르명', '공연지역명', '출연진내용', '제작진내용', '기획제작사명', '원작자명', '극작가명', '수상실적', '관람연령', '아동공연 여부', '축제 여부', '내한공연 여부', '오픈런 여부', '판매시작일시', '판매종료일시','단독판매여부'  
43 - 21 = 22

In [ ]:
# 드랍할 열 이름 리스트
columns_to_drop = ['공연회차','공연시작일자', '공연종료일자', '소요시간',
                   '장르명', '세부장르명', '공연지역명', '출연진내용',
                   '제작진내용', '기획제작사명', '원작자명', '극작가명',
                   '수상실적', '관람연령', '아동공연 여부', '축제 여부',
                   '내한공연 여부', '오픈런 여부', '판매시작일시', '판매종료일시',
                   '단독판매여부']

# 열 삭제
df = df.drop(columns=columns_to_drop)

### 공연일시 변환 (년, 월, 일) / 주말 및 공휴일 여부 추가


22+5 = 27

In [ ]:
# 연, 월, 일 칼럼 생성
df['연도'] = df['공연일시'].dt.year
df['월'] = df['공연일시'].dt.month
df['일'] = df['공연일시'].dt.day

# 공휴일 데이터프레임 생성 (예시로 일부 공휴일만 포함)
holidays = pd.to_datetime([
    '2023-01-01',  # 신정
    '2023-01-21',  # 설날 연휴
    '2023-01-22',  # 설날
    '2023-01-23',  # 설날 연휴
    '2023-03-01',  # 삼일절
    '2023-05-05',  # 어린이날
    '2023-05-27',  # 부처님오신날
    '2023-06-06',  # 현충일
    '2023-08-15',  # 광복절
    '2023-09-28',  # 추석 연휴
    '2023-09-29',  # 추석
    '2023-09-30',  # 추석 연휴
    '2023-10-03',  # 개천절
    '2023-10-09',  # 한글날
    '2023-12-25'   # 성탄절
])

# 주말 여부 칼럼 생성
df['주말여부'] = df['공연일시'].dt.weekday >= 5

# 공휴일 여부 칼럼 생성
df['공휴일여부'] = df['공연일시'].dt.normalize().isin(holidays)

# 주말 또는 공휴일 여부 칼럼을 1과 0으로 변환
df['주말여부'] = df['주말여부'].astype(int)
df['공휴일여부'] = df['공휴일여부'].astype(int)
# 결과 출력
print(df)

         전송사업자코드      공연코드                공연일시      입장권고유번호  \
0       SE000201  PF443217 2023-10-01 14:00:00  T2274085680   
1       SE000201  PF443217 2023-10-01 14:00:00  T2274086480   
2       SE000201  PF443217 2023-10-01 14:00:00  T2274088440   
3       SE000201  PF443217 2023-10-01 14:00:00  T2274107230   
4       SE000201  PF443217 2023-10-01 14:00:00  T2274089460   
...          ...       ...                 ...          ...   
651279  SE000201  PF443217 2023-08-31 19:30:00  T2275880870   
651280  SE000201  PF443217 2023-08-31 19:30:00  T2275880871   
651281  SE000201  PF443217 2023-08-31 19:30:00  T2275914430   
651282  SE000201  PF443217 2023-08-31 19:30:00  T2275915770   
651283  SE000201  PF443217 2023-08-31 19:30:00  T2276028800   

                   예매/취소일시  예매/취소구분  예매/취소매수  예매/취소금액  예매/취소방식코드  결제수단코드  ...  \
0      2023-08-29 14:00:00        1        1   161500          1       2  ...   
1      2023-08-29 14:00:00        1        1   161500          1       2  ...   


### 불필요한 컬럼 삭제(9개)
'공연일시', '예매/취소일시' > 예매후남은일자 칼럼으로 합침  
'입장권고유번호' > 식별자의 역할 후 삭제  
'예매/취소구분' > 라벨링에 사용후 삭제  
'예매/취소매수' > 모두 1이므로 삭제  
'판매페이지URL > 삭제  
'예매일시','취소일시' > 라벨링에 사용후 삭제  
27 - 8 = 19

In [ ]:
# 드랍할 열 이름 리스트
columns_to_drop = ['공연일시', '예매/취소일시','입장권고유번호',
                   '예매/취소구분', '예매/취소매수' ,'판매페이지URL',
                   '예매일시','취소일시',]

# 열 삭제
df = df.drop(columns=columns_to_drop)

- 전처리 필요 칼럼  
   '판매좌석수' > '석' 제거후 사용(0과 1은 결측치로 보고 결측치는 평균 대체)

In [ ]:
import numpy as np

# '판매좌석수' 열을 문자열로 변환
df['판매좌석수'] = df['판매좌석수'].astype(str)

# '석'으로 끝나는 경우에만 '석' 제거
df.loc[df['판매좌석수'].str.endswith('석', na=False), '판매좌석수'] = df['판매좌석수'].str.replace('석', '')

# 문자를 정수로 변환
df['판매좌석수'] = pd.to_numeric(df['판매좌석수'], errors='coerce')

# 판매좌석수 칼럼에서 1인 값을 결측치로 대체
df['판매좌석수'] = df['판매좌석수'].replace(1, np.nan)

# 판매좌석수 칼럼에서 0인 값을 결측치로 대체
df['판매좌석수'] = df['판매좌석수'].replace(0, np.nan)

# 결측치를 평균값으로 대체
mean_value = df['판매좌석수'].mean()
df['판매좌석수'] = df['판매좌석수'].fillna(mean_value)

# 평균값으로 대체한 후, 정수로 변환
df['판매좌석수'] = df['판매좌석수'].astype(int)

# 결과 출력
print(df)


         전송사업자코드      공연코드  예매/취소금액  예매/취소방식코드  결제수단코드  할인금액  할인종류코드    장당금액  \
0       SE000201  PF443217   161500          1       2     0      99  161500   
1       SE000201  PF443217   161500          1       2     0      99  161500   
2       SE000201  PF443217   161500          1       2     0      99  161500   
3       SE000201  PF443217   161500          1       2     0      99  161500   
4       SE000201  PF443217   161500          1       2     0      99  161500   
...          ...       ...      ...        ...     ...   ...     ...     ...   
651279  SE000201  PF443217    76000          2       2     0      99   76000   
651280  SE000201  PF443217    76000          2       2     0      99   76000   
651281  SE000201  PF443217    80000          2       2     0      99   80000   
651282  SE000201  PF443217   133000          2       2     0      99  133000   
651283  SE000201  PF443217    80000          3       2     0      99   80000   

        성별    연령                       

### '성별', '연령' > 성별 결측치 최빈값 대체, 연령은 범주로 연령은 삭제

19+3-1=21

In [ ]:
# 성별 칼럼에서 0인 값을 결측치로 대체
df['성별'] = df['성별'].replace(0, np.nan)

# 최빈값으로 결측치 대체
mode_value = df['성별'].mode()[0]
df['성별'] = df['성별'].fillna(mode_value)

# 연령 칼럼에서 0인 값을 결측치로 대체
df['연령'] = df['연령'].replace(0, np.nan)

# 결측치를 평균값으로 대체
mean_value = df['연령'].mean()
df['연령'] = df['연령'].fillna(mean_value)

# 현재 연도를 기준으로 연령 계산 (2023년 기준)
current_year = 2023
df['연령'] = current_year - df['연령']

# 새로운 연령대 칼럼 생성
df['노인'] = df['연령'] >= 69
df['청년'] = (df['연령'] >= 7) & (df['연령'] <= 24)
df['성인'] = (df['연령'] < 69) & (df['연령'] > 24)

# Boolean을 1과 0으로 변환
df['노인'] = df['노인'].astype(int)
df['청년'] = df['청년'].astype(int)
df['성인'] = df['성인'].astype(int)

# 연령 칼럼 삭제
df = df.drop(columns=['연령'])


In [ ]:
# 장당금액 칼럼의 고유값 출력
unique_values = df['장당금액'].unique()
print("장당금액 칼럼의 고유값들:\n", unique_values)

장당금액 칼럼의 고유값들:
 [  161500   170000   140000   133000   104500   110000   119000   191400
    88000    80000    64000    76000    56000        0    98000   164400
    77000   153000    85000    70000    55000    40000   102000    25000
    52000   136000    48000   126000    66000   112000    84000    82500
   144500    60000 30000000]


In [ ]:
# '장당가격'이 1,700,000보다 큰 행을 필터링
large_price_df = df[df['장당금액'] > 170000]

# 해당되는 데이터 개수 출력
num_large_price_records = large_price_df.shape[0]  # 또는 len(large_price_df)
print(f"'장당금액'이 170,000보다 큰 데이터 개수: {num_large_price_records}")

'장당금액'이 170,000보다 큰 데이터 개수: 1217


### 이상치로 분류된 1,217개 데이터 제거

In [ ]:
# '장당금액'이 170,000보다 큰 데이터는 이상치로 보고 제거
df = df[df['장당금액'] <= 170000]

'좌석등급' > 좌석등급별 원핫인코딩 > 결제금액과 할인금액 이용
결제금액,장당금액,할인금액

In [ ]:
import numpy as np

# 0이 아닌 '장당금액' 값들의 비율을 계산
non_zero_values = df[df['장당금액'] != 0]['장당금액']
value_counts = non_zero_values.value_counts(normalize=True)

# 0인 값을 대체하는 함수
def replace_zero_value():
    return np.random.choice(value_counts.index, p=value_counts.values)

# 0인 값 대체
df.loc[df['장당금액'] == 0, '장당금액'] = df.loc[df['장당금액'] == 0, '장당금액'].apply(lambda x: replace_zero_value())

# 결과 확인
print(df['장당금액'].value_counts())


장당금액
170000    78557
161500    44165
140000    33131
153000    21500
133000    20251
110000    13060
126000    11064
80000     10869
88000      8709
104500     6846
64000      6697
119000     6512
76000      5522
85000      4251
98000      3102
77000      2395
56000      2264
70000      2093
136000     1513
52000      1503
55000       641
164400      620
40000       506
144500      443
112000      274
102000      269
25000       265
60000       161
84000        31
82500        19
66000         9
48000         2
Name: count, dtype: int64


### 좌석등급 전처리(공연장별 수정 필요)

R석(140000), OP석(140000), S석(100000), A석(70000)

In [ ]:
# 좌석등급1 칼럼 생성 및 조건에 따라 값 할당
def assign_seat_grade(row):
    if row['공연코드'] == 'PF443217':
        if 140000 < row['장당금액'] <= 170000:
            return 'VIP'
        elif 110000 < row['장당금액'] <= 140000:
            return 'R'
        elif 80000 < row['장당금액'] <= 110000:
            return 'S'
        elif row['장당금액'] <= 80000:
            return 'A'
    return None

df['좌석등급1'] = df.apply(assign_seat_grade, axis=1)

# 좌석등급 칼럼 삭제
df = df.drop(columns=['좌석등급'])

# 좌석등급1 칼럼명을 좌석등급으로 변경
df.rename(columns={'좌석등급1': '좌석등급'}, inplace=True)


# 결과 출력
print(df)

         전송사업자코드      공연코드  예매/취소금액  예매/취소방식코드  결제수단코드  할인금액  할인종류코드    장당금액  \
0       SE000201  PF443217   161500          1       2     0      99  161500   
1       SE000201  PF443217   161500          1       2     0      99  161500   
2       SE000201  PF443217   161500          1       2     0      99  161500   
3       SE000201  PF443217   161500          1       2     0      99  161500   
4       SE000201  PF443217   161500          1       2     0      99  161500   
...          ...       ...      ...        ...     ...   ...     ...     ...   
651279  SE000201  PF443217    76000          2       2     0      99   76000   
651280  SE000201  PF443217    76000          2       2     0      99   76000   
651281  SE000201  PF443217    80000          2       2     0      99   80000   
651282  SE000201  PF443217   133000          2       2     0      99  133000   
651283  SE000201  PF443217    80000          3       2     0      99   80000   

         성별  판매좌석수  ...  취소여부    연도   월

<ipython-input-19-34f248819b28>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['좌석등급1'] = df.apply(assign_seat_grade, axis=1)


## 3. 공연코드, 예매/취소금액 드랍

21-2=19

In [ ]:
# 공연코드, 예매/취소금액 드랍
df = df.drop(columns = ['공연코드','예매/취소금액'])

## 4. 명목 변수 라벨 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 라벨 인코딩 적용
label_encoder = LabelEncoder()

# 라벨 인코딩 결과를 저장할 딕셔너리
label_mapping = {}

# 각 칼럼에 대해 라벨 인코딩 적용
for column in ['예매/취소방식코드', '전송사업자코드', '할인종류코드', '결제수단코드','좌석등급']:
    df[column] = label_encoder.fit_transform(df[column])

    # 각 칼럼의 라벨 인코딩 매핑 결과 저장
    label_mapping[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# 라벨 인코딩 매핑 결과 출력
for column, mapping in label_mapping.items():
    print(f"'{column}' 컬럼의 라벨 매핑 결과:")
    for original_value, encoded_value in mapping.items():
        print(f"  {original_value} -> {encoded_value}")
    print("\n")

# # 데이터프레임을 Excel 파일로 저장
# df.to_excel('/content/drive/MyDrive/KOPIS/블루스퀘어/전체데이터(라벨인코딩).xlsx', index=False, engine='openpyxl')


'예매/취소방식코드' 컬럼의 라벨 매핑 결과:
  1 -> 0
  2 -> 1
  3 -> 2
  4 -> 3
  99 -> 4


'전송사업자코드' 컬럼의 라벨 매핑 결과:
  SE000105 -> 0
  SE000201 -> 1
  SE000307 -> 2
  SE000331 -> 3


'할인종류코드' 컬럼의 라벨 매핑 결과:
  3 -> 0
  99 -> 1


'결제수단코드' 컬럼의 라벨 매핑 결과:
  1 -> 0
  2 -> 1
  4 -> 2
  5 -> 3
  99 -> 4


'좌석등급' 컬럼의 라벨 매핑 결과:
  A -> 0
  R -> 1
  S -> 2
  VIP -> 3




In [ ]:
# 결과 출력
print("라벨 인코딩 결과:\n", df)

# 데이터프레임을 Excel 파일로 저장
df.to_excel('/content/drive/MyDrive/KOPIS/블루스퀘어/전체데이터(라벨인코딩).xlsx', index=False, engine='openpyxl')

라벨 인코딩 결과:
         전송사업자코드  예매/취소방식코드  결제수단코드  할인금액  할인종류코드    장당금액   성별  판매좌석수  예매후남은일자  \
0             1          0       1     0       1  161500  2.0   1309     33.0   
1             1          0       1     0       1  161500  2.0   1309     33.0   
2             1          0       1     0       1  161500  1.0   1309     33.0   
3             1          0       1     0       1  161500  2.0   1309     32.0   
4             1          0       1     0       1  161500  1.0   1309     33.0   
...         ...        ...     ...   ...     ...     ...  ...    ...      ...   
651279        1          1       1     0       1   76000  2.0   1309      0.0   
651280        1          1       1     0       1   76000  2.0   1309      0.0   
651281        1          1       1     0       1   80000  1.0   1309      0.0   
651282        1          1       1     0       1  133000  1.0   1309      0.0   
651283        1          2       1     0       1   80000  2.0   1309      0.0   

        취소여부   

## 5. 11월 19일 공연 데이터 분리 (시뮬레이션 목적)

In [ ]:
# 테스트 데이터프레임: 연도가 2023, 월이 11, 일이 19인 데이터
test_df = df[(df['연도'] == 2023) & (df['월'] == 11) & (df['일'] == 19)]

# 트레이닝 데이터프레임: 나머지 데이터
train_df = df[~((df['연도'] == 2023) & (df['월'] == 11) & (df['일'] == 19))]

# 결과 출력
print("테스트 데이터프레임:\n", test_df)
print("\n트레이닝 데이터프레임:\n", train_df)

테스트 데이터프레임:
         전송사업자코드  예매/취소방식코드  결제수단코드  할인금액  할인종류코드    장당금액   성별  판매좌석수  예매후남은일자  \
202772        1          0       1     0       1  119000  2.0   1307     54.0   
202773        1          1       1     0       1  161500  1.0   1307     54.0   
202774        1          1       1     0       1  161500  2.0   1307     54.0   
202775        1          1       1     0       1  161500  2.0   1307     54.0   
202776        1          0       1     0       1  170000  1.0   1307     54.0   
...         ...        ...     ...   ...     ...     ...  ...    ...      ...   
206490        2          1       1     0       1  170000  2.0  35642      0.0   
206491        2          1       1     0       1  133000  2.0  35642      0.0   
206492        2          1       1     0       1  161500  2.0  35642      0.0   
206496        2          1       1     0       1  161500  2.0  35635      0.0   
206497        2          2       0     0       1  133000  2.0  35635      0.0   

        취소여부  

In [ ]:
# 데이터프레임을 Excel 파일로 저장
test_df.to_excel('/content/drive/MyDrive/KOPIS/블루스퀘어/테스트_데이터(라벨인코딩).xlsx', index=False, engine='openpyxl')
# 데이터프레임을 Excel 파일로 저장
train_df.to_excel('/content/drive/MyDrive/KOPIS/블루스퀘어/트레이닝_데이터(라벨인코딩).xlsx', index=False, engine='openpyxl')